<a href="https://colab.research.google.com/github/kimgu11/NLP_2023/blob/main/example/maze_runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import random
import time

In [1]:
class Maze:  #미로 생성 코드
    view_delay_sec = 1.0

    def __init__(self, height, width, ratio=0.5):
        self.height = height
        self.width = width
        self.start_row, self.start_col = 1, 1
        self.end_row, self.end_col = height, width

        self._maze_init()
        self._fill(ratio)

    def _maze_init(self):
        hwall = ['#'] * (self.width + 2)
        self.maze = [hwall]
        for row in range(1, self.height + 1):
            self.maze.append(['#'] + [' '] * self.width + ['#'])
        self.maze.append(hwall)

    def _fill(self, ratio):
        empty_slots = []
        for row in range(1, self.height + 1):
            for col in range(1, self.width + 1):
                if (row, col) not in [(1, 1), (self.height, self.width)]:
                    empty_slots.append((row, col))

        n_blk = int(self.height * self.width * ratio)
        inner_walls = random.choices(empty_slots, k=n_blk)
        for row, col in inner_walls:
            self.maze[row][col] = '#'

    def in_range(self, row, col):
        return 0 <= row <= self.height and 0 <= col <= self.width

    def is_empty(self, row, col):
        return self.maze[row][col] == ' '

    def is_wall(self, row, col):
        return self.in_range(row, col) and self.maze[row][col] == '#'

    def save(self, filename):
        try:
            with open(filename, "wb") as f:
                pickle.dump(self, f)
            return True
        except IOError:
            return False

    @classmethod
    def load(cls, filename):
        try:
            with open(filename, "rb") as f:
                obj = pickle.load(f)
            return obj
        except IOError:
            return None

    def __repr__(self):
        repr = []
        for row in self.maze:
            repr.append(''.join(row))
        return '\n'.join(repr)

    def is_valid_path(self, path):
        def _neighbors(r1, c1, r2, c2):
            return (r1 - r2, c1 - c2) in [(1, 0), (0, 1), (-1, 0), (0, -1)]

        if len(path) < 1 or path[0] != (1, 1):
            return False

        for i in range(1, len(path)):
            if not self.is_empty(*path[i]):
                return False

            if not _neighbors(*path[i - 1], *path[i]):
                return False

        if path[-1] != (self.height, self.width):
            return False

        return True

    def view_path(self, path):
        if not self.is_valid_path(path):
            print("view_path(): Not a valid path")
            return

        print("view_path() starts from (1, 1)... ")
        print()
        time.sleep(Maze.view_delay_sec)

        cnt = 0
        for row, col in path:
            cnt += 1
            print('current =', (row, col), ', length =', cnt)
            self.maze[row][col] = '@'
            print(self)
            print()
            time.sleep(Maze.view_delay_sec)

        print("view_path() finished at ({}, {}), length = {}".format(self.height, self.width, len(path)))
        print()

        for row, col in path:
            self.maze[row][col] = ' '